<a href="https://colab.research.google.com/github/krish04shah/Material-And-Energy-Balances/blob/main/HW2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import quad

In [ ]:
R = 8.314 #J/mol*K
Tc = 647.096 #K
Pc = 22.064e6 #Pa
w = 0.348
b = 1/8 * R * Tc / Pc

In [ ]:
def calc_abes(T, eos='SRK'):
    Tr=T/Tc
    s={'vdw':0, 'SRK':1}[eos]
    e={'vdw':0, 'SRK':0}[eos]
    omega={'vdw':1/8, 'SRK':0.08664}[eos]
    psi={'vdw':27/64, 'SRK':0.42748}[eos]
    alpha={'vdw':1,'SRK': (1. + (0.48+1.574*w-0.176*w**2)*(1-Tr**0.5))**2 }[eos]
    a=psi*alpha*(R**2)*(Tc**2)/Pc
    b=omega*R*Tc/Pc
    return a,b,e,s


In [ ]:
def cubic_P(V,T,eos = 'SRK'):

  a,b,e,s = calc_abes(T,eos)

  return ((R*T)/(V-b)) - (a/((V + e*b)*(V + s*b)))

In [ ]:
ONETHIRD = 1/3
def cubic_roots(a,b,c):
    Q = (a**2-3*b)/9.
    R = (2*a**3 - 9*a*b + 27*c)/54
    if R**2 < Q**3:
        theta = np.arccos(R/Q**1.5)
        return -2*np.sqrt(Q)*np.cos(np.array([theta/3,  (theta-2*np.pi)/3, (theta+2*np.pi)/3])) - a/3
    else:
        A = -(R+(R**2-Q**3)**0.5)**ONETHIRD
        B = 0 if A == 0 else Q/A
        return (A+B) - a/3

In [ ]:
def cubic_V(P,T,eos = 'SRK'):

  a,b,e,s = calc_abes(T,eos)

  beta = (b*P)/(R*T)
  q = P/(b*R*T)

  z = cubic_roots( (e+s)*beta - beta -1., e*s*beta**2. - (e+s)*beta**2. - (e+s)*beta + q*beta, -e*s*beta**3. - e*s*beta**2. - q*beta**2.)

  return (z*R*T)/P

In [ ]:
def cubic_v(P,T,eos='SRK'):
    a,b,e,s = calc_abes(T,eos)
    beta=b*P/R/T
    q=a/(b*R*T)
    zs=cubic_roots( (e+s)*beta - beta -1.,
                   e*s*beta**2 - (e+s)*beta**2 - (e+s)*beta + q*beta,
                   -e*s*beta**3 - e*s*beta**2 - q*beta**2)
    return zs*R*T/P

In [ ]:
cubic_v(10e6, 600)

array([4.26115084e-05, 7.93845409e-05, 3.76843951e-04])

In [ ]:
def maxwell_areas(P,T,eos='SRK'):
    vl, vm, vg = cubic_v(P,T,eos)
    return quad(lambda v: P - cubic_P(v,T,eos), vl, vm)[0], quad(lambda v: cubic_P(v,T,eos)-P, vm, vg)[0]

In [ ]:
V=np.logspace(np.log10(1.001*b), -2, 1000)

In [ ]:
fig = make_subplots()

for T in np.arange(275, 900, 25):
  fig.add_scatter(x=V, y=cubic_P(V, T,eos='SRK'),mode='lines',name=f'{T} K')
fig.update_xaxes(type='log')
fig.update_yaxes(range=[-1e7,1e8])
fig.update_layout(width=800,height=600,template='plotly_dark')

In [ ]:
maxwell_areas(13e6,600)

(287.8738049182429, 208.01190331050867)

In [ ]:
maxwell_areas(91e3,373)

(9154.889447297845, 9143.0263427625)